In [1]:
%%capture
!pip install matatika==1.13.0
!pip install wheel
!pip install pandas==1.4.3 numpy==1.26.4
!pip install weasyprint==58.1
!pip install matatika-iplotter==1.3.0
!pip install git+https://github.com/Matatika/htmlcharts.git@v0.1.0

In [2]:
import pandas as pd
import os
from datetime import datetime, timedelta, date
from htmlcharts.bar import bar_chart
from matatika.library import MatatikaClient
from matatika.types import DataFormat
from iplotter import ChartJSPlotter
from weasyprint import HTML, CSS

In [3]:
from matatika.library import MatatikaClient

endpoint_url = os.getenv('ENDPOINT_URL')
workspace_id = os.getenv('WORKSPACE_ID')

# Attempt to use the pipeline 'AUTH_TOKEN'
auth_token = os.getenv('AUTH_TOKEN')
# On 401 use CLIENT_ID and CLIENT_SECRET to refresh
client_id = os.getenv('NOTEBOOK_CLIENT_ID')
client_secret = os.getenv('NOTEBOOK_CLIENT_SECRET')

client = MatatikaClient(auth_token=auth_token, client_id=client_id, client_secret=client_secret, endpoint_url=endpoint_url, workspace_id=workspace_id)
plotter = ChartJSPlotter()

In [4]:
# Test failure tables data
new_failing_tests = client.fetch("elementary/export_new_failing_tests")
new_failing_tests_df = pd.DataFrame(new_failing_tests)

all_failing_tests = client.fetch("elementary/export_all_failing_tests")
all_failing_tests_df = pd.DataFrame(all_failing_tests)


# Chart data
chart_dataset_id = 'elementary/dashboard/bars/test-failures'

dataset_information = client.get_dataset(chart_dataset_id)

chart_data = client.fetch(chart_dataset_id)

In [5]:
# Dates logic

current_date = datetime.now()
seven_days_ago_date =  current_date - timedelta(days=6)

current_date_str = current_date.strftime("%B %d, %Y")
seven_days_ago_str = seven_days_ago_date.strftime("%B %d, %Y")

date_range = f"{seven_days_ago_str} - {current_date_str}"

last_seven_days_dates = []

for i in range(6, -1, -1):
    date_str = (current_date - timedelta(days=i)).strftime("%Y-%m-%d")
    last_seven_days_dates.append(date_str)

In [6]:
# Data points for display in email

issues_today = 0
issues_yesterday = 0
yesterday_comparison = 0
for data_point in chart_data:

    if datetime.strptime(data_point["Date"], '%Y-%m-%d').date() == datetime.now().date():
        issues_today = int(data_point["Failures"])

    if datetime.strptime(data_point["Date"], '%Y-%m-%d').date() == datetime.now().date() - timedelta(days = 1):
        issues_yesterday = int(data_point["Failures"])

issue_comparison =  issues_today - issues_yesterday

if issue_comparison == 0 and issues_today == 0 and issues_yesterday == 0:
    issue_percentage = 0
elif issues_today > 0 and issues_yesterday == 0:
    issue_percentage = 100
else:
    issue_percentage = round(issue_comparison / issues_yesterday * 100)


issue_percentage_arrow = ''

if issue_percentage > 0:
    issue_percentage_color = 'red'
    issue_percentage_arrow = '&uarr;'
    
elif issue_percentage < 0:
    issue_percentage_color = 'green'
    issue_percentage_arrow = '&darr;'
else: 
    issue_percentage_color = 'darkgrey'
    

In [7]:
# Set up email html

html = """
<head>
    <style>
        .content {
            max-width: 700px;
            text-align: left;
            margin: auto;
        }

        .bar-display {
            font: 18px sans-serif;
            text-align: center;
            color: rgb(120, 120, 120);
            position: relative;
            margin-top: auto;
        }

        .label {
            display: block;
            color: black;
            font-size: 14px;
        }

        .flex-charts {
            align-items: center; 
            justify-content: center;
        }

        .m2 {
            margin-bottom: 2px;
            margin-top: 2px;
        }

        .mb5 {
            margin-bottom: 5px;
        }

        .mb30 {
            margin-bottom: 30px;
        }

        .mt0 {
            margin-top: 0px;
        }

        .mt10 {
            margin-top: 10px;
        }

        .mt15 {
            margin-top: 15px;
        }

        .mt30 {
            margin-top: 30px;
        }

        .pl10 {
            padding-left: 10px;
        }

        .pr10 {
            padding-right: 10px;
        }

        hr {
            border: 0;
            height: 1px;
            border-top: 1px solid rgba(0, 0, 0, 0.2);
            border-bottom: 1px solid rgba(255, 255, 255, 0.2);
        }

        .dataframe td {
            overflow-wrap: anywhere;
        }

        .dataframe {
            table-layout: auto;
            width: 100%;
        }

        .dataframe td {
            text-align: left;
        }

    </style>
</head>
<div class="content">
"""

html += f'''
<div class="header">
<img src="https://app.matatika.com/assets/images/utility/matatika.png" style="width: 80px; float: left;"/>
<div style="text-align: right;">
<h1 class="mb5">Daily Report</h1>
<p class="mt10">{date_range}</p>
</div>
</div>
<hr class="mb30 mt30">
<div>
<div class="pr10">
<h3 class="mt0 mb5">Total Issues</h3>
<h1 class="issue-count m2">{issues_today}</h1>
<p class="m2">vs. yesterday</p>
<p class="m2" style="color: {issue_percentage_color}">{issue_percentage}% {issue_percentage_arrow}</p>
</div>
</div>
<a href="https://app.matatika.com/l/{workspace_id}/dashboard" class="mt15">View Dashboard</a>
<div class="flex-charts">
<div class="chart-holder">
'''

In [8]:
# Create chart for email

# Add data points for each day in the last 7 days missing from the data, set failures to 0
for d in last_seven_days_dates:
    dates_in_chart_data = []
    for data_point in chart_data:
        dates_in_chart_data.append(data_point["Date"])
    if d not in dates_in_chart_data:
        new_data_point = {}
        new_data_point['Date'] = d
        new_data_point['Failures'] = '0'
        chart_data.append(new_data_point)

data_dict = {}

# Make all failures values ints
for data_point in chart_data:
    data_dict[data_point['Date']] = int(data_point["Failures"])

# Order the dates
sorted_dict = dict(sorted(data_dict.items(), key=lambda x: x[0]))

chart_data_dict = {}

# Change the date keys into 'Mon', 'Tue'...
for date_str, value in sorted_dict.items():
    date = datetime.strptime(date_str, '%Y-%m-%d')
    chart_data_dict[date.strftime('%a')] = value
  
html += bar_chart(chart_data_dict, bar_color="rgba(255, 204, 203, 0.8)", chart_width=700, bar_height=350)

html += '''
</div>
</div>
<hr class="mb30 mt30">
<div>
<h3>New Issues</h3>
'''


In [9]:
# Create tables to list tests in email
html_output = new_failing_tests_df.to_html(index=False, header=False)
if new_failing_tests:
    html += html_output.replace('style="text-align: right;"', 'border: 0;"').replace('border="1"', '')
else:
    html += '<p>No new issues</p>'

html += "<h3>All Issues</h3>"

html_output = all_failing_tests_df.to_html(index=False)
if all_failing_tests:
    html += html_output.replace('style="text-align: right;"', 'style="border: 0;"').replace('border="1"', '')
else:
    html += '<p>No issues</p>'

html += '</div></div>'

In [10]:
# Write html out to file
with open("email_template.html", 'w') as f:
    f.write(html)